In [11]:
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
import horizon_api
import json
import os
import subprocess

MAPA_TELES = "telesa/"

# Simulacija nebesnih teles

## Prenos nebesnih teles
Poženite spodnji blok da lahko ustvatite datoteke za izbrana telsa na datum ki ga vnesete.

In [ ]:
slovar_teles = horizon_api.search_body()
datum = input("zapišite izbran datum v formatu yyyy-mm-dd")

text_ime_telesa = widgets.Text(description="Ime telesa")
dropdown_izbira = widgets.Dropdown(
    options=[],
    description='Izbira telesa:',
)
out = widgets.Output(layout={'border': '1px solid black', 'padding': "10px"})

# populira dropdown z najdenimi ključi za iskan niz
def najdi_telo(t):
    ime_telesa = str(t.value).capitalize()
    
    if ime_telesa in slovar_teles.keys():   najdeno = {ime_telesa: slovar_teles[ime_telesa]}
    else:                                   najdeno = horizon_api.search_body(ime_telesa)
    
    out.clear_output(); dropdown_izbira.value=None
    if not najdeno: 
        with out: print("Žal nismo našli nobeno telo z tem ključem...") 
    else:
        with out: print("Izberite želeno telo s seznama") 
        dropdown_izbira.options = najdeno.keys()

# na disku ustvari datoteko ki predstavlja izbrano telo
def ustvari_telo(t):
    if t['new'] != t['old'] and t['new']:
        name = t['new']
        json_data = horizon_api.get_body_json(slovar_teles[name], datum, name=name)

        with open(f"{MAPA_TELES}telo_{name}.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=4)
        out.clear_output()

        if json_data['mass'] == 0 or json_data["radius"] == 0 or len(json_data["x_vec"]) == 0 or len(json_data["v_vec"]) == 0:
            with out: print("Telo ustvarjeno! (manjkajo ključni podatki)")
        else:
            with out: print("Telo ustvarjeno!") 
        

text_ime_telesa.on_submit(najdi_telo)
dropdown_izbira.observe(ustvari_telo, names="value")

display(text_ime_telesa)
display(dropdown_izbira)
display(out)

## Primerjava nebesnih teles po polmeru in velikosti
Prvi blok prenesena telesa parsira in jih srani v seznam, nasledji blok pa za izbrana nariše grafike, ki jih lahko uporabnik prenese.

In [ ]:
# naložimo vsa prenesena telesa in priredimo barve telesom glede na naraščajoč polmer
prenesena_telesa = []

cm = plt.get_cmap('gist_rainbow')
for datoteka in list(os.listdir(MAPA_TELES)):
    with open(os.path.join(MAPA_TELES, datoteka)) as f:
        prenesena_telesa.append(json.load(f))

prenesena_telesa = sorted(prenesena_telesa, key=lambda x: x["radius"])
for i in range(len(prenesena_telesa)):
    prenesena_telesa[i]["color"] = cm(i/len(prenesena_telesa))

In [ ]:
dropdown_grafike = widgets.SelectMultiple(
    options=[(telo["name"], i) for i, telo in enumerate(prenesena_telesa)],
    description='Izbira teles za prikaz grafik',
    rows = 10,
)

btn_narisi = widgets.Button(description='Ustvari grafiki')

# nariše grafiki za polmere in mase
def ustvari_primerjavo(t):
    # izbiro uredimo po velikosti polmerov
    izbira = [prenesena_telesa[i] for i in dropdown_grafike.value]
    izbira = sorted(izbira, key=lambda x: x["radius"])

    # narišemo grafiko velikosti
    graf_velikosti, graf_velikosti_axis = plt.subplots()
    x = 0
    for telo in izbira:
        x += telo["radius"]
        circ = plt.Circle((x,0),telo["radius"], color=telo["color"], label=telo["name"])
        x += telo["radius"]

        graf_velikosti_axis.add_patch(circ)
        graf_velikosti_axis.legend()

    graf_velikosti.set_figwidth(15)
    graf_velikosti_axis.autoscale(); graf_velikosti_axis.set_xticks([])
    graf_velikosti_axis.set_aspect('equal', adjustable='box')

    # narišemo grafiko mase
    graf_mase, graf_mase_axis = plt.subplots()
    bars = graf_mase_axis.bar([telo["name"] for telo in izbira], [math.log(telo["mass"]) for telo in izbira])
    for i, bar in enumerate(bars): bar.set_color(izbira[i]["color"])

    graf_mase.set_figwidth(15); graf_mase_axis.autoscale()
    graf_mase_axis.set_ylabel("20*log(kg)")

display(dropdown_grafike); display(btn_narisi)
btn_narisi.on_click(ustvari_primerjavo)

## 3d prikaz gibanja nebesnih teles
Spodnji blok simulira gibanje prenesenih teles in jih prikaže v 3d prostoru. 

In [42]:
# pridobi input za izbrana telesa, centrirano gleda na zadnje telo v seznamu
def pridobi_input(telesa, obdobje_casa, st_korakov):
    stanja_teles = [f"{telo['mass']} {telo['x_vec'][0]} {telo['x_vec'][1]} {telo['x_vec'][2]} {telo['v_vec'][0]} {telo['v_vec'][1]} {telo['v_vec'][2]}" for telo in telesa]
    return f"6.674 {obdobje_casa} {st_korakov} {len(stanja_teles)} {' '.join(stanja_teles)}"

def simuliraj(telesa, obdobje_casa, st_korakov):
    inp = pridobi_input(telesa, obdobje_casa, st_korakov)
    
    # poženemo simulacijo
    proces_simulacije = subprocess.Popen(
        ["./simulacija"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )
    stdout = proces_simulacije.communicate(input=inp.encode("utf-8"))[0].decode("utf-8").split("\n")

    # parsiramo output v lepšo obliko za vsako telo posebaj
    podatki_simulacije = [[] for _ in range(len(telesa))]
    for i in range(st_korakov):
        for j in range(len(telesa)):
            podatki_simulacije[j].append(tuple(
                map(float,
                    stdout[i*len(telesa)+j].strip().split()
                )
            ))

    return podatki_simulacije